----

*Problemas:*

- Os arquivos `[DGroh,MRocha,ALeles]` estão faltando muitas linhas... vai gerar uma inconsistência muito forte, visto que só terão 2 arquivos "preenchidos"

- Os arquivos `[Viní16,LCoelho]` possuem muita inconsistência (valores acima de 10), o `Viní16` inclusive, possui linhas que **não** podem existir, vide "10" em na competência _segurança_ 

# Análise sobre o perfil dos alunos formandos/formados do curso de Engenharia de Computação

**Resumo**

Este arquivo irá conter todos os dados referentes à pesquisa realizada.

----


## Obtendo a grade

Obtendo a grade do curso de computação 2015, com pré e có requisitos

In [1]:
# Módulo usado para obter a grade (do banco ou realiza raspagem)
from modules.Scrapping import get_grid
# Dataframe
import pandas as pd
# Lista os arquivos no diretório
from os import listdir

In [2]:
def grid(grid):
    GraphPre, GraphCo = get_grid(grid)
    # Print grid
    # only_grid(GraphPre.nodes(), GraphPre.edges(), GraphCo.edges())
    return GraphPre, GraphCo


def read_csv():
    # Get form's infos
    infos = []
    sheetsDir = "../assets/sheets"
    for file in listdir(sheetsDir):
        tmpDF = pd.read_csv(f"{sheetsDir}/{file}").fillna(0)
        # Remove two last lines
        tmpDF = tmpDF[:-2]
        # Renaming columns
        header = tmpDF.columns.to_list()
        header[0] = 'Periodo'
        header[1] = 'Sigla'
        header[2] = 'Nome da disciplina'
        header[-1] = 'Somatório'
        tmpDF.columns = header
        # Removing unecessary columns
        tmpDF.drop(['Periodo', 'Nome da disciplina',
                    'Somatório'], axis=1, inplace=True)
        # Turn initials into index
        tmpDF.set_index('Sigla', inplace=True)

        # Append this dataframe to info vec
        infos.append(tmpDF)

    # Return a list of dataframes
    return infos

In [3]:
pre, co = grid('0192015')

### Obtêm os pesos das competências

Faz a leitura dos arquivos csv

In [4]:
infos = read_csv()

Realiza a média dos arquivos de competências

In [5]:
# Juntando os arquivos num único dataframe
out = pd.concat(infos)

# Realizando 
out = out.groupby(out.index)
out = out.mean()

## Transformando a grade em uma relação de competências

- Cada competência gera um único _grafo_, assim, é necessário propagar os nós para todos os grafos e atribuir os vértices de acordo com a entrada dos alunos nos arquivos de *competências*

In [6]:
# Obtendo uma cópia do grafo de pré requisito (este grafo possui o período de cada disciplina)
nodes = pre.copy()

# Removendo os vértices (conexões, que nates eram os pré requisitos)
nodes.remove_edges_from(list(nodes.edges()))

In [7]:
# Obtêndo o nome de cada coluna (será utilizada no grafo novo)
competencias = out.columns.to_list()

In [8]:
# Removendo os nós que não constam naquela planilha
nos_para_remocao = [no for no in list(nodes.nodes()) if no not in out.index.to_list()]
nodes.remove_nodes_from(nos_para_remocao)

In [9]:
# Variável que irá iterar sobre os períodos
def get_periodo():
    """
    Função que computa o período atual
    """
    p = 0
    while p<10:
        p+= 1
        yield p

In [10]:
def debug(msg,*args):
    """
    Print a debug message

    Parameters
    ----------
    msg: string
    """
    printString = msg
    if args:
        vec = list(map(lambda x: str(x), list(args)))
        vec = ', '.join(vec)
        printString += ': ' + vec
    print(f'[DEBUG] -> {printString}')



In [11]:
def get_materias(periodo):
    """
    Obtêm todas a lista de todas as siglas de matérias do período

    Parameters
    ----------
    periodo: número que indica o período atual entre [1,10]

    Returns
    -------
    Uma lista contendo todas as siglas das matérias deste período
    """
    global nodes
    return [sigla for sigla,data in nodes.nodes(data=True) if data['period']==str(periodo)]



def check_competencia(materias_atuais, competencia):
    """
    Verifica se tem algum valor para esta competência neste periodo

    Parameters
    ----------
    materias_atuais: lista de materias no periodo atual
    competencia: competencia a ser analizada

    Returns
    -------
    True para caso não tenha uma disciplina no período atual que possua esta competência
    """
    global out
    return sum([out.loc[i][competencia] for i in materias_atuais]) == 0



def get_peso_competencia(materias_atuais, competencia, materia):
    """
    FIXME: Peso errado? Não, o peso das arestas é referente ao período do vértice de onde sai, se só tiver 1 matéria, irá ser peso 1 para qualquer aresta.

    Obtêm o peso da disciplina normalizado pelo peso de disciplinas no semestre, i.e., o máximo que uma competência pode ter no semestre é 1 ou 100%

    Parameters
    ----------
    materias_atuais: lista de materias no periodo atual
    competencia: competencia a ser analizada
    materia: materia que se deseja obter o peso da competencia para este periodo

    Returns
    -------
    Um float arredondado à duas casas decimais
    """
    global out
    total = sum([out.loc[i][competencia] for i in materias_atuais])

    # debug(f'get_peso_competencia({materia},  {materias_atuais}) -> Total {total}, Matéria {out.loc[materia][competencia]}\n')
    return round(out.loc[materia][competencia]/total, 2)

## Gerando os grafos

In [12]:
# Instância que irá ter todos os objetos de grafos indexados pelas competências
grafos = {}

# Obtêm as matérias do período seguinte (ou do próximo) que tiver este requisito, gerando conexões entre essas disciplinas
for competencia in competencias:
    # Instancia um novo grafo (baseado no grafo que já possui os nós) para essa competência
    grafos[competencia] = nodes.copy()

     # Itera essa competência para os próximos períodos
    for periodo in get_periodo():
        # Obtêm a lista de todas as matérias do período atual
        materias_periodo_atual = get_materias(periodo)
        # Se maior igual à zero, nenhuma das matérias do período possui valor para essa competência
        if check_competencia(materias_periodo_atual,competencia):
            # portanto, pode pular esta disciplina
            continue

        # Lista para armazenar todos os nós de matérias que possuem o mesmo requisito nos próximos períodos
        materias_herdeiras = []

        # Itera sobre os próximos períodos
        prox_periodo = periodo
        while prox_periodo < 10:
            prox_periodo += 1
            # Obtêm a lista de todas as matérias do período atual
            materias_proximo_periodo = get_materias(prox_periodo)
            # Verifica se neste período possui alguma matéria que herda aquela competência
            if check_competencia(materias_proximo_periodo, competencia):
                continue
            else:
                # Caso encontre alguma matéria neste período coloca ela na lista de materias que irão herdar a competência
                for i in materias_proximo_periodo:
                    if out.loc[i][competencia] > 0:
                        materias_herdeiras.append(i)
                # Força a saída deste loop
                break

        # Para cada matéria atual que possui a competência e cada matéria futura que irá herdar, realiza a "junção"
        for materia_atual in materias_periodo_atual:
            # Verifica se a matéria possui esta competência
            if not out.loc[materia_atual][competencia]:
                continue
            for materia_herdeira in materias_herdeiras:
                peso = get_peso_competencia(materias_periodo_atual,competencia,materia_atual)
                grafos[competencia].add_edge(
                    materia_atual,
                    materia_herdeira, 
                    weight=peso
                )



## Plot dos grafos de competências

In [13]:
# Realiza o plot do grafo/grade
from modules.Plot import weighted_graph, only_grid

In [14]:
# Itera sobre cada grafo de competencia
for competencia in grafos.keys():
    # Gera um plot para cada um com o peso saindo de cada matéria
    weighted_graph(
        grafos[competencia].nodes(),
        list(grafos[competencia].edges(data=True)),
        competencia.title().replace(' ','').replace('.','')
    )


## Buscando as notas através do bot de raspagem

Mandando o `spyder` rodar com os parâmetros abaixo:
```bash
curl -u 9a357fcfff6341378238837dcce40bdf: https://app.scrapinghub.com/api/run.json \
  -d project=460296 \
  -d spider=history \
  -d user='NUMERO_CPF' \
  -d pswd='SENHA_SIGAA' \
  -d course='NÃO_É_ÚTIL_AQUI' 
```

You can get the **Api key** [here](https://app.scrapinghub.com/account/apikey)

#### Obtendo as credenciais do arquivo json

In [15]:
# Loads the credentials file
from json import load

# Load credentials json
credentials = None
with open('credentials.json') as file:
    credentials = load(file)

In [25]:
from scrapinghub import ScrapinghubClient
from time import sleep as Wait


# Run job
apikey = '9a357fcfff6341378238837dcce40bdf'
client = ScrapinghubClient(apikey)
project = client.get_project(460296)
job = project.jobs.run('history', 
    job_args={
        'user': credentials['user.login'],
        'pswd': credentials['user.senha'],
        # 'course':'0192015'
})

# Check if job is running
job_state = 'running'
while job_state != 'finished':
    # Check for job state at each 1.5 min
    Wait(60*1.5)
    # Get job state from scrapyhub
    job_state = job.finish()

Os dados obtidos estão em um formato do tipo
```javascript
{
    "CourseName": "ENGENHARIA DE COMPUTAÇÃO/ICT - Itabira - BACHARELADO - MT",
    "CurrentYearPeriod": "2020.2",
    "StartYearPeriod": "2016.1",
    "RA": "2016001942",
    "Semester": {
      "2020.2": {
        "ECOI32.1": {
          "Name": "CIRCUITOS INTEGRADOS ANALÓGICOS",
          "Score": "6,7",
          "Fouls": "0",
          "Situation": "APROVADO",
          "Class": "01",
          "Hours": "32h"
        },
        ...
      },
      ...
    }
}
```

## Obtendo somente uma relação de notas para as matérias

Realiza a média no cenário que há bomba.

In [32]:
periodos = {}
# Obtendo as notas
for items in job.items.iter():
    periodos = items[b'Semester']

In [33]:
notas = {}
# Convertendo os dados
for periodo in periodos:
    periodoVetor = periodos[periodo]
    for materia in periodoVetor:
        materiaUtf8 = materia.decode('utf-8')
        # Se encontrar uma disciplina que tenha um ponto, trata-se de um bloco
        if materiaUtf8[-2] == '.':
            # Então ignora, pois ela já está contabilizada na disciplina final (sem o ponto)
            continue
        

        score = periodoVetor[materia][b'Score']
        # Ignora as matérias que ainda estão sendo feitas
        if b'--' in score:
            continue
        
        # Converte para float
        score = float(score.decode('utf-8').replace(',','.'))
        
        # Verifica se o aluno já fez a matéria
        if materiaUtf8 in notas:
            # Verifica se já repetiu mais de uma vez
            if type(notas[materiaUtf8]) is list:
                # Se já tiver repetido mais de uma vez, adiciona ao vetor
                notas[materiaUtf8].append(score)
            else:
                # Primeira vez que repetiu
                notas[materiaUtf8] = [notas[materiaUtf8], score]
        else:
            # Primeira vez que fez a matéria
            notas[materiaUtf8] = score


# Para cada elemento no dicionário de notas, verifica se repetiu, caso tenha, realiza média
for materia in notas:
    if type(notas[materia]) is list:
        total = sum(notas[materia])
        numero_elementos = len(notas[materia])
        notas[materia] = round(total/numero_elementos, 2)

In [ ]:
# from pprint import pprint
# pprint(notas)

### Andando no grafo e propagando os valores

$\forall$ Competência $\to$ verifica se existe a nota, caso exista, propaga

Informações sobre a biblioteca podem ser encontradas [aqui](https://networkx.org/documentation/stable//reference/classes/digraph.html#methods)


In [34]:
def get_nota(materia:str, peso:float, acumulado:float) -> float:
    # Nota do aluno
    global notas
    # Caso o aluno não tenha feito a matéria ainda, propaga o acumulado pelo peso
    if materia not in notas:
        return round(acumulado * peso, 3)
    # Caso já tenha feito a matéria, calcula pelo peso e retorna mais o acumulado
    return round((notas[materia]/10 + acumulado)*peso , 3)

In [35]:
import sys
# A função poderá ser chamada recursivamente 1000x (default)
sys.setrecursionlimit(1000)

# DFS
def dfs_walk(grafo, materia, acumulado=0) -> float:
    """
    A recursive walk
    """
    global notas
    total = 0

    # Anda sobre os filhos
    for filho in grafo.neighbors(materia):
        # Obtém o peso da aresta que manda para o filho
        peso = grafo[materia][filho]['weight']
        # Obtém a nota (acumulada) que será enviada para o filho
        acumulado = get_nota(materia, peso, acumulado)
        # Caminha para este filho
        total += dfs_walk(grafo, filho, acumulado)
    else:
        # Não possui filhos (última da grade com essa competência)
        total = get_nota(materia, 1, acumulado)
    
    # Retorna o valor acumulado (dos filhos e até ela)
    return total

In [36]:
import sys

def errprint(msg):
    sys.stderr.write('Error: ' + msg)

In [37]:
# Dicionário que irá conter o valor sobre cada competência
comparacao_nota:dict = {}

# Itera sobre as competências
for competencia,grafo in grafos.items():
    # ∀ competência, encontra a primeira matéria que possui ela
    edges = list(grafo.edges())
    # Uma vez que o grafo foi montado em ordem cronológica, não há necessidade
    # de busca.
    if not edges:
        errprint('Não tem ligações')
        continue
    fst_materia = edges[0][0]
    # Em seguida, anda no seu grafo e obtêm o valor iterado sobre as notas
    resultado = dfs_walk(grafo, fst_materia)
    comparacao_nota[competencia] = resultado

Error: Não tem ligações

In [38]:
for competencia, resultado in comparacao_nota.items():
    print(competencia, resultado)

Matemática e física 0.937
Lógica, algoritmos, teoria da comp,  estruras de dados. 0.809
Linguagens e paradigmas. 1.8
PAA 1.89
Configurar plataformas para softwares e serviços. 1.8
Arquiteturas de computadores 2.49
Segurança de sis. de comp. 1.64
Engenharia de software 1.28
Inteligência artificial 1.56
Sistemas microprocessados 0.918
Redes de computadores 1.6
Software para sistemas de comunicação 0.417
Conhecimento em sistemas de automação  1.26
Gerenciar projetos e sistemas de computação 1.42
Engenharia-econômica 1.26
Compreender e resolver problemas 1.55
Autoaprendizado 0.876
Criatividade e Inovação 0.809
Comunicação oral e escrita 0.898
Língua inglesa 4.9
Empreender e exercer liderança 1.065
Trabalho em equipe 1.256


Normalizando a saída com base no maior valor

In [66]:
comparacao_nota_sem_normalizar = comparacao_nota.copy()

maior_valor = max(comparacao_nota.values())

for competencia, resultado in comparacao_nota.items():
    comparacao_nota[competencia] = round(resultado/maior_valor, 2)

In [67]:
# Valores normalizados
for competencia, resultado in comparacao_nota.items():
    print(competencia, resultado)

Matemática e física 0.19
Lógica, algoritmos, teoria da comp,  estruras de dados. 0.17
Linguagens e paradigmas. 0.37
PAA 0.39
Configurar plataformas para softwares e serviços. 0.37
Arquiteturas de computadores 0.51
Segurança de sis. de comp. 0.33
Engenharia de software 0.26
Inteligência artificial 0.32
Sistemas microprocessados 0.19
Redes de computadores 0.33
Software para sistemas de comunicação 0.09
Conhecimento em sistemas de automação  0.26
Gerenciar projetos e sistemas de computação 0.29
Engenharia-econômica 0.26
Compreender e resolver problemas 0.32
Autoaprendizado 0.18
Criatividade e Inovação 0.17
Comunicação oral e escrita 0.18
Língua inglesa 1.0
Empreender e exercer liderança 0.22
Trabalho em equipe 0.26


## Insere os valores do mercado

In [59]:
from random import random

# Colocando valores randomicos
saida_mercado = comparacao_nota.copy()
for competencia, _ in saida_mercado.items():
    saida_mercado[competencia] = round(random(),2)
    print(competencia + '\t' + str(saida_mercado[competencia]))

Matemática e física	0.61
Lógica, algoritmos, teoria da comp,  estruras de dados.	0.27
Linguagens e paradigmas.	0.41
PAA	0.61
Configurar plataformas para softwares e serviços.	0.11
Arquiteturas de computadores	0.58
Segurança de sis. de comp.	0.23
Engenharia de software	0.6
Inteligência artificial	0.24
Sistemas microprocessados	0.94
Redes de computadores	0.82
Software para sistemas de comunicação	0.43
Conhecimento em sistemas de automação 	0.83
Gerenciar projetos e sistemas de computação	0.48
Engenharia-econômica	0.71
Compreender e resolver problemas	0.88
Autoaprendizado	0.9
Criatividade e Inovação	0.02
Comunicação oral e escrita	0.2
Língua inglesa	0.92
Empreender e exercer liderança	0.45
Trabalho em equipe	0.07


Converte para um vetor e faz o produto escalar com base nos valores do mercado

In [68]:
vetor_mercado = list(saida_mercado.values())
vetor_aluno = list(comparacao_nota.values())


In [69]:
import numpy as np

proximidade = round(np.dot(vetor_mercado,vetor_aluno),2)

print(f'Os valores estão com uma proximidade de {proximidade}%')

Os valores estão com uma proximidade de 3.74%


----

# ToDO:
- [x] Gerar uma visualização de grafo para cada competência (Próxima reunião)
- [x] Buscar notas e fazer as médias
- [x] Iterar sobre cada competência, gerando um valor relativo às notas
- [x] Juntar todos os valores relativo às competências e comparar com o outro vetor

--
- [ ] Tentar normalizar a saída das arestas
- [x] Tentar comparar com um valor aleatório
- [x] Fazer uma tela para o framework no [Figma] ou indo diretamente para sua [view]
- [ ] Tentar pegar mais algumas notas com algum aluno
- [ ] Comparar com a resposta do mercado

--

<br/>

**To search for**:

- Redes siamesas
- Ominilog


[figma]: https://www.figma.com/file/6qlPm5NjjzjRyGJUYWntpv/Projeto-Artigo?node-id=0%3A1
[view]: https://www.figma.com/proto/6qlPm5NjjzjRyGJUYWntpv/Projeto-Artigo?node-id=18%3A43&scaling=min-zoom


----
# Tentando refazer o processo normalizando a planilha de competências